# Modelling

In [4]:
import pandas as pd

# Load the data
df = pd.read_csv('Data/clean_data.csv')

df

,sentiment,raw_comm,nouns_comm
0,neutral,Technopolis plans to develop in stages an area...,Technopolis stages area meters order companies...
1,negative,The international electronic industry company ...,industry company tens employees facility layof...
2,positive,With the new production plant the company woul...,production plant company capacity increase dem...
3,positive,According to the company 's updated strategy f...,company strategy years term sales growth range...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,FINANCING growth strategy circuit boards PCBs
...,...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...,Share prices rebound bank stocks weakness
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,beer sales per cent litres beer sales cent litres
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,profit mn mn vessel sales gain mn
4843,negative,Net sales of the Paper segment decreased to EU...,sales segment mn quarter mn quarter profit ite...


### Split the data

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['sentiment'], random_state=42)

# Display the class distribution in the training set before downsampling
print("Training set class distribution before downsampling:")
print(train_df['sentiment'].value_counts())

# Display the class distribution in the test set
print("Test set class distribution:")
print(test_df['sentiment'].value_counts())

Training set class distribution before downsampling:
neutral     2302
positive    1091
negative     483
Name: sentiment, dtype: int64
Test set class distribution:
neutral     576
positive    272
negative    121
Name: sentiment, dtype: int64


### Up sampling

In [6]:
# Determine the size of the largest class in the training set
max_class_size = train_df['sentiment'].value_counts().max()

# Upsample each sentiment class in the training set
train_positive = train_df[train_df['sentiment'] == 'positive'].sample(max_class_size, replace=True, random_state=42)
train_negative = train_df[train_df['sentiment'] == 'negative'].sample(max_class_size, replace=True, random_state=42)
train_neutral = train_df[train_df['sentiment'] == 'neutral'].sample(max_class_size, replace=True, random_state=42)

# Combine the upsampled dataframes
train_df_upsampled = pd.concat([train_positive, train_negative, train_neutral])

# Shuffle the combined dataframe to mix the classes
train_df_upsampled = train_df_upsampled.sample(frac=1, random_state=42).reset_index(drop=True)

# Display the class distribution after downsampling in the training set
print("Training set class distribution after downsampling:")
print(train_df_upsampled['sentiment'].value_counts())


Training set class distribution after downsampling:
negative    2302
positive    2302
neutral     2302
Name: sentiment, dtype: int64


### TF-IDF Vectorization

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fill NaN values in 'nouns_comm' and 'raw_comm' columns with an empty string
train_df_upsampled['nouns_comm'] = train_df_upsampled['nouns_comm'].fillna('')
train_df_upsampled['raw_comm'] = train_df_upsampled['raw_comm'].fillna('')
test_df['nouns_comm'] = test_df['nouns_comm'].fillna('')
test_df['raw_comm'] = test_df['raw_comm'].fillna('')

# TF-IDF Vectorization for 'nouns_comm' and 'raw_comm'
tfidf_vectorizer_nouns = TfidfVectorizer()
tfidf_vectorizer_raw = TfidfVectorizer()

X_train_nouns = tfidf_vectorizer_nouns.fit_transform(train_df_upsampled['nouns_comm'])
X_test_nouns = tfidf_vectorizer_nouns.transform(test_df['nouns_comm'])

X_train_raw = tfidf_vectorizer_raw.fit_transform(train_df_upsampled['raw_comm'])
X_test_raw = tfidf_vectorizer_raw.transform(test_df['raw_comm'])

y_train = train_df_upsampled['sentiment']
y_test = test_df['sentiment']


## Naive Bayes Model

### Train the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Train Naive Bayes models for both 'nouns_comm' and 'raw_comm'
nb_nouns = MultinomialNB()
nb_raw = MultinomialNB()   

# Fit the models
nb_nouns.fit(X_train_nouns, y_train)
nb_raw.fit(X_train_raw, y_train)

# Predict on the training set
y_train_pred_nouns = nb_nouns.predict(X_train_nouns)
y_train_pred_raw = nb_raw.predict(X_train_raw)

# Predict on the test set
y_test_pred_nouns = nb_nouns.predict(X_test_nouns)
y_test_pred_raw = nb_raw.predict(X_test_raw)


### Compute models' accuracy

In [ ]:
# Evaluate the Naive Bayes model on the test set ('nouns_comm')
print("\nTest Set - Naive Bayes Model on 'nouns_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_nouns))
print("Classification Report:\n", classification_report(y_test, y_test_pred_nouns))

# Evaluate the Naive Bayes model on the test set ('raw_comm')
print("\nTest Set - Naive Bayes Model on 'raw_comm'")
print("Accuracy:", accuracy_score(y_test, y_test_pred_raw))
print("Classification Report:\n", classification_report(y_test, y_test_pred_raw))


### Add predictions in the test_df

In [ ]:
test_df['NB_pred_noun'] = y_test_pred_nouns
test_df['NB_pred_raw'] = y_test_pred_raw

test_df

## FNN

### FNN with nouns comm

#### Create the FNN architecture

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def create_fnn_model(input_dim, layers=1, nodes=32, learning_rate=0.001, dropout_rate=0.5, l2_reg=0.01):
    """
    Creates a feedforward neural network model with L2 regularization and dropout.
    
    Args:
        input_dim (int): Number of input features.
        layers (int): Number of hidden layers.
        nodes (int): Number of nodes per layer.
        learning_rate (float): Learning rate for the optimizer.
        dropout_rate (float): Dropout rate to prevent overfitting.
        l2_reg (float): L2 regularization strength.

    Returns:
        model: Compiled Keras model.
    """
    model = Sequential()
    model.add(Dense(nodes, input_dim=input_dim, activation='relu', kernel_regularizer=l2(l2_reg)))
    
    # Add hidden layers with dropout and L2 regularization
    for _ in range(layers - 1):
        model.add(Dense(nodes, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
    
    # Output layer with 3 nodes for the 3 classes
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

#### Create the Grid Search and fit the model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Function to create the KerasClassifier model for grid search
def create_fnn_model(input_dim, layers, nodes, learning_rate, dropout_rate=0.5, l2_reg=0.01):
    model = Sequential()
    model.add(Dense(nodes, input_dim=input_dim, activation='relu'))
    
    # Add hidden layers
    for _ in range(layers - 1):
        model.add(Dense(nodes, activation='relu'))
        model.add(Dropout(dropout_rate))
    
    # Output layer with 3 nodes for 3 classes (assuming classification with 3 labels)
    model.add(Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Wrapping the model for scikit-learn compatibility
def build_keras_classifier(input_dim, layers, nodes, learning_rate, dropout_rate=0.5, l2_reg=0.01):
    return KerasClassifier(
        model=create_fnn_model, input_dim=input_dim, layers=layers, 
        nodes=nodes, learning_rate=learning_rate, dropout_rate=dropout_rate, 
        l2_reg=l2_reg, verbose=0
    )

# Define hyperparameters for grid search
param_grid = {
    'layers': [3, 5, 7],
    'nodes': [128, 256, 512],
    'epochs': [20],
    'batch_size': [32]
}

# Build the model
input_dim = X_train_nouns.shape[1]
model = build_keras_classifier(input_dim=input_dim, layers=1, nodes=32, learning_rate=0.001)

# EarlyStopping and ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0.001, 
    patience=10,  
    verbose=0, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=5,  
    verbose=0, 
    min_lr=0.0001
)

# Perform Grid Search with EarlyStopping and ReduceLROnPlateau callbacks
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

# Pass callbacks when fitting the model
grid_result = grid.fit(X_train_nouns, y_train, callbacks=[early_stopping, reduce_lr])


2024-09-08 16:14:53.653818: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 16:14:53.658086: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 16:14:53.659958: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-08 16:14:53.661740: I tensorflow/core/platform/cpu_featu

Epoch 1/20
Epoch 1/20
Epoch 1/20
Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4037 - loss: 3.2202
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.3772 - loss: 3.2335
107/144 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.3601 - loss: 5.4756Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.3642 - loss: 3.2228
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.3715 - loss: 4.7886
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.5450 - loss: 1.0917
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5198 - loss: 1.1126
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4968 - loss: 1.1186 
 76/144 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5420 - loss: 1.0745Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.5506 - loss: 1.0713
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6131 - loss: 0.9981
Epoch 4/20
144/144 ━━━━━━━━━━━━━━━━━

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/stepep - accuracy: 0.8127 - loss: 0.74
104/144 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8129 - loss: 0.7374Epoch 1/20
137/144 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8134 - loss: 0.7393

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

 76/144 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.3269 - loss: 1.0988Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.3274 - loss: 1.0988
Epoch 15/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.3348 - loss: 1.0988
Epoch 9/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 44s 83ms/step - accuracy: 0.3308 - loss: 15.0854
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.3189 - loss: 1.0989
Epoch 16/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.3405 - loss: 1.0990
Epoch 10/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.3239 - loss: 1.0989
Epoch 17/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.3399 - loss: 1.0986
Epoch 11/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 12s 80ms/step - accuracy: 0.3421 - loss: 1.0987
Epoch 3/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.3232 - loss: 1.0989
Epoch 12/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.3255 - loss: 1.0987
Epoch 18/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 25

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


144/144 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.3279 - loss: 1.0987
Epoch 17/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.3267 - loss: 1.0990
Epoch 5/20
 34/144 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3407 - loss: 1.0985Epoch 1/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.3387 - loss: 1.09867
Epoch 18/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.3232 - loss: 1.0989
Epoch 19/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.3155 - loss: 1.0989
Epoch 20/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 13s 91ms/step - accuracy: 0.3234 - loss: 1.0988
Epoch 6/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 50s 93ms/step - accuracy: 0.3250 - loss: 15.0929
Epoch 2/20
144/144 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.3266 - loss: 1.0987
144/144 ━━━━━━━━━━━━━━━━━━━━ 7s 48ms/step - accuracy: 0.3229 - loss: 1.0992
Epoch 3/20
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/stepep - accuracy: 0.3359 - loss: 1.098
144/144 ━━━━━━━━━━━━━━━━━━━━ 16s 59ms/step - accuracy: 0.33

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: module 'keras.src.activations' has no attribute 'get'

#### Save the model

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

best_model_nouns = grid_result.best_estimator_

# Access the underlying Keras model
keras_model = best_model_nouns.model_

# Save the model in the recommended format
keras_model_nouns.save('FNN_nouns.keras')

# Load the model from the saved file
best_model_nouns = load_model('FNN_nouns.keras')

# Convert Sentiment labels to numerical values
label_encoder = LabelEncoder()
y_test_encoded = label_encoder.fit_transform(y_test)

# Evaluate the loaded model with the encoded labels
best_model_nouns.evaluate(X_test_nouns, y_test_encoded)


#### Print the mean cross validation accuracy per combination 

In [ ]:
# Summarize the results from the grid search
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Extract the mean and standard deviation of test scores, and the corresponding parameters
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

# Print the results in a readable format
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


#### Retrain the best model to store the history

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import LabelEncoder

# Retrieve the best hyperparameters from GridSearchCV
best_params = grid_result.best_params_

# Create the final model using the best parameters
best_model = create_fnn_model(
    input_dim=X_train_nouns.shape[1],
    layers=best_params['layers'],
    nodes=best_params['nodes'],
    learning_rate=best_params['learning_rate']
)

# Model Summary
print(best_model.summary())

# EarlyStopping to stop training when validation loss has not improved
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0.001, 
    patience=10, 
    verbose=1, 
    restore_best_weights=True
)

# ReduceLROnPlateau to reduce the learning rate when validation loss has stopped improving
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=10, 
    verbose=1, 
    min_lr=0.0005
)

# ModelCheckpoint to save the model after every epoch
checkpoint = ModelCheckpoint(
    'best_model.keras', 
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Fit the model using the encoded labels
history = best_model.fit(
    X_train_nouns, y_train_encoded,  # <-- Use encoded labels here
    epochs=best_params['epochs'],  # Use the best number of epochs from the grid search
    batch_size=best_params['batch_size'],  # Use the best batch size from the grid search
    validation_split=0.2,  # Using 20% of the training data as validation data
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)


#### Plot the history of the loss function and accuracy

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    """
    Plots the training and validation loss and accuracy.

    Args:
        history: Keras History object returned by model.fit().
    """
    # Plot training & validation accuracy values
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()

# Plot the training and validation loss/accuracy using the history object
plot_training_history(history)


### Predict on the test set with the best model

In [ ]:
# Retrieve the best model from GridSearchCV
best_model_nouns = grid_result.best_estimator_

# Make predictions on the test set
y_test_pred = best_model_nouns.predict(X_test_nouns)


#### Compute metrics of the predictions

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
test_accuracy = best_model_nouns.score(X_test_nouns, y_test)
print("Test set accuracy: {:.4f}".format(test_accuracy))

# Generate a classification report
print(classification_report(y_test, y_test_pred))

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)


# CHECK TO SEE IF THE MODEL IS BETTER WITH OVERFITING MEASURES BEFORE DOING THE SAME FOR RAW COMM

## FNN with raw comms

### Train and plot with raw comments

In [ ]:
# Perform Grid Search with EarlyStopping and ReduceLROnPlateau on raw comments
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_raw, y_train, callbacks=[early_stopping, reduce_lr])

# Retrieve the best model from GridSearchCV
best_model_raw = grid_result.best_estimator_

# Access the underlying Keras model
keras_model_raw = best_model_raw.model_

# Save the best model trained on raw comments
keras_model_raw.save('fnn_raw_model.keras')

# Retrain the best model on the full training set and store the training history
history = best_model_raw.model_.fit(
    X_train_raw, y_train, 
    epochs=best_model_raw.get_params()['epochs'], 
    batch_size=best_model_raw.get_params()['batch_size'],
    validation_split=0.2,  # Use 20% of the training data for validation
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot the training and validation accuracy and loss using the history
plot_training_history(history)


### Metrics with raw comments

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Retrieve the best model from GridSearchCV
best_model_raw = grid_result.best_estimator_

# Make predictions on the test set (use X_test_raw instead of X_test_nouns)
y_test_pred = best_model_raw.predict(X_test_raw)

# Evaluate the model on the test set (using raw comments)
test_accuracy = best_model_raw.score(X_test_raw, y_test)
print("Test set accuracy: {:.4f}".format(test_accuracy))

# Generate a classification report
print(classification_report(y_test, y_test_pred))

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)
